In [31]:
import pandas as pd
import numpy as np
import datetime
from datetime import timedelta
from time import strftime
import plotly.express as px
import glob
import os
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [32]:
# Missing Data from November 8th, 2020 to January 11st, 2021
files = os.path.join("C:\\Users\\aaliy\\OneDrive\\Documents\\GitHub\\energyResearch\\", "bingData*.csv")
files = glob.glob(files)

bingData = pd.concat(map(pd.read_csv, files))
bingData

,START_TIME,END_TIME,BUILDING_ID,TYPE,VALUE
0,2019-01-01 00:10:07,2019-01-01 01:10:07,MH,E,8.669
1,2019-01-01 00:10:07,2019-01-01 01:10:07,LA,E,6.808
2,2019-01-01 00:10:07,2019-01-01 01:10:07,KH,E,6.475
3,2019-01-01 00:10:07,2019-01-01 01:10:07,JH,E,8.623
4,2019-01-01 00:10:07,2019-01-01 01:10:07,HE,E,5.472
...,...,...,...,...,...
115590,2021-12-31 22:10:09,2021-12-31 23:10:09,HSB,E,36.000
115591,2021-12-31 22:10:09,2021-12-31 23:10:09,HSB,H,NaN
115592,2021-12-31 22:10:09,2021-12-31 23:10:09,HD,H,2300.000
115593,2021-12-31 22:10:09,2021-12-31 23:10:09,HD,E,NaN


In [33]:
# DATA CLEANING PROCESS
# Isolate to just Type "E" which refers to Electricity
bingData = bingData[bingData['TYPE'] == 'E']

# Add Time Information to bingData Data Frame
bingData['time'] = pd.to_datetime(bingData['START_TIME'])
bingData['Time'] = pd.to_datetime(bingData['time']).dt.time
bingData['Date'] = pd.to_datetime(bingData['time']).dt.date
bingData['Week'] = bingData['time'].dt.strftime("%U")
bingData['Day'] = bingData['time'].dt.strftime("%j")
bingData['Day'] = pd.to_numeric(bingData['Day'], errors='coerce')
bingData['Weekday'] = bingData['time'].dt.strftime("%w")
bingData['Hour'] = pd.to_datetime(bingData['time']).dt.hour

bingData

#Single the data to just the Fall Semester
bingData['Week'] = pd.to_numeric(bingData['Week'], errors='coerce')
bingData = bingData[bingData['Week'] >= 31]

C:\Users\aaliy\AppData\Local\Temp/ipykernel_40156/3064461973.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\aaliy\AppData\Local\Temp/ipykernel_40156/3064461973.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\aaliy\AppData\Local\Temp/ipykernel_40156/3064461973.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/panda

In [61]:
unionData = bingData[bingData['BUILDING_ID'] == 'UU']
LHData = bingData[bingData['BUILDING_ID'] == 'LH']
libData = bingData[bingData['BUILDING_ID'] == 'LH']

#Found Max Demand for Each Day
idx = unionData.groupby(['Date'])['VALUE'].transform(max) == unionData['VALUE']
unionData = unionData[idx].drop_duplicates(['Date'], keep='first', ignore_index=True)
unionData

idx = LHData.groupby(['Date'])['VALUE'].transform(max) == LHData['VALUE']
LHData = LHData[idx].drop_duplicates(['Date'], keep='first', ignore_index=True)
LHData

idx = libData.groupby(['Date'])['VALUE'].transform(max) == libData['VALUE']
libData = libData[idx].drop_duplicates(['Date'], keep='first', ignore_index=True)


#unionData = unionData.drop(unionData.index[650:],0)
#unionData
unionData

,START_TIME,END_TIME,BUILDING_ID,TYPE,VALUE,time,Time,Date,Week,Day,Weekday,Hour
0,2019-08-04 15:10:10,2019-08-04 16:10:10,UU,E,406.24,2019-08-04 15:10:10,15:10:10,2019-08-04,31,216,0,15
1,2019-08-05 14:10:02,2019-08-05 15:10:02,UU,E,562.52,2019-08-05 14:10:02,14:10:02,2019-08-05,31,217,1,14
2,2019-08-06 14:10:01,2019-08-06 15:10:01,UU,E,643.64,2019-08-06 14:10:01,14:10:01,2019-08-06,31,218,2,14
3,2019-08-07 13:10:06,2019-08-07 14:10:06,UU,E,630.52,2019-08-07 13:10:06,13:10:06,2019-08-07,31,219,3,13
4,2019-08-08 11:10:08,2019-08-08 12:10:08,UU,E,609.63,2019-08-08 11:10:08,11:10:08,2019-08-08,31,220,4,11
...,...,...,...,...,...,...,...,...,...,...,...,...
396,2021-12-27 10:10:08,2021-12-27 11:10:08,UU,E,203.26,2021-12-27 10:10:08,10:10:08,2021-12-27,52,361,1,10
397,2021-12-28 09:10:07,2021-12-28 10:10:07,UU,E,202.31,2021-12-28 09:10:07,09:10:07,2021-12-28,52,362,2,9
398,2021-12-29 10:10:08,2021-12-29 11:10:08,UU,E,213.36,2021-12-29 10:10:08,10:10:08,2021-12-29,52,363,3,10
399,2021-12-30 10:10:09,2021-12-30 11:10:09,UU,E,199.91,2021-12-30 10:10:09,10:10:09,2021-12-30,52,364,4,10


In [97]:
#px.scatter(x=libData['Date'],y=libData['VALUE'])
print(np.min(libData['VALUE']))
print(np.max(libData['VALUE']))
print(np.mean(libData['VALUE']))
print(np.median(libData['VALUE']))
px.box(libData,x='VALUE', title='Library Data Box Plot')


46.143
416.366
186.10063341645883
185.21


In [38]:
#How to deal with that one month of missing data? -> Maybe remove 2020 because covid
#do sklearn multivariable regression + calculate error

In [90]:
#Imported weather Data
weather = pd.read_csv('weatherData2019-2021.csv')
weather = pd.DataFrame({'DateTime':weather['DATE'], 'Temp':weather['HourlyDryBulbTemperature'],'Humidity':weather['HourlyRelativeHumidity']}).dropna()
weather

weather['time'] = pd.to_datetime(weather['DateTime'])
weather['Time'] = pd.to_datetime(weather['time']).dt.time
weather['Date'] = pd.to_datetime(weather['time']).dt.date
weather['Week'] = weather['time'].dt.strftime("%U")
weather['Day'] = weather['time'].dt.strftime("%j")
weather['Day'] = pd.to_numeric(weather['Day'], errors='coerce')
weather['Weekday'] = weather['time'].dt.strftime("%w")
weather['Hour'] = pd.to_datetime(weather['time']).dt.hour

#Single to fall semester
weather['Week'] = pd.to_numeric(weather['Week'], errors='coerce')
weather = weather[weather['Week'] >= 31]

#Found Max Temp for Each Day
idx = weather.groupby(['Date'])['Temp'].transform(max) == weather['Temp']
weatherTemp = weather[idx].drop_duplicates(['Date'], keep='first', ignore_index=True)


C:\Users\aaliy\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning:

Columns (42,43,50,52,74,76,90) have mixed types.Specify dtype option on import or set low_memory=False.



In [91]:
#Found Max Humidity for Each Day
idx = weather.groupby(['Date'])['Humidity'].transform(max) == weather['Humidity']
weatherHumid = weather[idx].drop_duplicates(['Date'], keep='first', ignore_index=True)

In [95]:
#Combine Max Temp + Max Humidity
weatherMax = pd.DataFrame({'Temp':weatherTemp['Temp'],'Humidity':weatherHumid['Humidity']})
weatherMax

,Temp,Humidity
0,77.0,100.0
1,79.0,87.0
2,82.0,100.0
3,77.0,100.0
4,78.0,100.0
...,...,...
450,31.0,96.0
451,38.0,100.0
452,38.0,100.0
453,41.0,100.0


In [54]:
#MANUAL SOLUTION
#Remove November 8th 2020 to January 11th 2021 (Weather Data index 677-741)
weather_data_adjusted = weather[(weather.index < 677) | (weather.index > 741)]
weather_data_adjusted
#Realized there are other data gaps - maybe try linear interpolation later

#DYNAMIC SOLUTION to remove the data from both data frames if missing data
#Merge johnson and weather data
mergedData = pd.merge(weather, LHData, on=['Date'], how='inner')
mergedData

,DateTime,Temp,Humidity,time_x,Time_x,Date,Week_x,Day_x,Weekday_x,Hour_x,...,END_TIME,BUILDING_ID,TYPE,VALUE,time_y,Time_y,Week_y,Day_y,Weekday_y,Hour_y
0,2019-08-04T12:53:00,77.0,58.0,2019-08-04 12:53:00,12:53:00,2019-08-04,31,216,0,12,...,2019-08-04 18:10:01,LH,E,179.711,2019-08-04 17:10:01,17:10:01,31,216,0,17
1,2019-08-05T14:53:00,79.0,31.0,2019-08-05 14:53:00,14:53:00,2019-08-05,31,217,1,14,...,2019-08-05 18:10:03,LH,E,171.752,2019-08-05 17:10:03,17:10:03,31,217,1,17
2,2019-08-06T14:53:00,82.0,53.0,2019-08-06 14:53:00,14:53:00,2019-08-06,31,218,2,14,...,2019-08-06 15:10:01,LH,E,216.926,2019-08-06 14:10:01,14:10:01,31,218,2,14
3,2019-08-07T12:29:00,77.0,66.0,2019-08-07 12:29:00,12:29:00,2019-08-07,31,219,3,12,...,2019-08-07 13:10:08,LH,E,220.631,2019-08-07 12:10:08,12:10:08,31,219,3,12
4,2019-08-08T12:53:00,78.0,58.0,2019-08-08 12:53:00,12:53:00,2019-08-08,31,220,4,12,...,2019-08-08 16:10:06,LH,E,217.943,2019-08-08 15:10:06,15:10:06,31,220,4,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
396,2021-12-27T23:53:00,31.0,92.0,2021-12-27 23:53:00,23:53:00,2021-12-27,52,361,1,23,...,2021-12-28 00:10:09,LH,E,58.443,2021-12-27 23:10:09,23:10:09,52,361,1,23
397,2021-12-28T11:53:00,38.0,70.0,2021-12-28 11:53:00,11:53:00,2021-12-28,52,362,2,11,...,2021-12-28 01:10:03,LH,E,58.055,2021-12-28 00:10:03,00:10:03,52,362,2,0
398,2021-12-29T20:53:00,38.0,93.0,2021-12-29 20:53:00,20:53:00,2021-12-29,52,363,3,20,...,2021-12-29 14:10:01,LH,E,59.199,2021-12-29 13:10:01,13:10:01,52,363,3,13
399,2021-12-30T20:53:00,41.0,100.0,2021-12-30 20:53:00,20:53:00,2021-12-30,52,364,4,20,...,2021-12-30 12:10:05,LH,E,56.524,2021-12-30 11:10:05,11:10:05,52,364,4,11


In [41]:
#Occupancy value for holidays, weekends, school days

In [55]:
mergedData['Weekday_y'] = pd.to_numeric(mergedData['Weekday_y'], errors='coerce')
mergedData

,DateTime,Temp,Humidity,time_x,Time_x,Date,Week_x,Day_x,Weekday_x,Hour_x,...,END_TIME,BUILDING_ID,TYPE,VALUE,time_y,Time_y,Week_y,Day_y,Weekday_y,Hour_y
0,2019-08-04T12:53:00,77.0,58.0,2019-08-04 12:53:00,12:53:00,2019-08-04,31,216,0,12,...,2019-08-04 18:10:01,LH,E,179.711,2019-08-04 17:10:01,17:10:01,31,216,0,17
1,2019-08-05T14:53:00,79.0,31.0,2019-08-05 14:53:00,14:53:00,2019-08-05,31,217,1,14,...,2019-08-05 18:10:03,LH,E,171.752,2019-08-05 17:10:03,17:10:03,31,217,1,17
2,2019-08-06T14:53:00,82.0,53.0,2019-08-06 14:53:00,14:53:00,2019-08-06,31,218,2,14,...,2019-08-06 15:10:01,LH,E,216.926,2019-08-06 14:10:01,14:10:01,31,218,2,14
3,2019-08-07T12:29:00,77.0,66.0,2019-08-07 12:29:00,12:29:00,2019-08-07,31,219,3,12,...,2019-08-07 13:10:08,LH,E,220.631,2019-08-07 12:10:08,12:10:08,31,219,3,12
4,2019-08-08T12:53:00,78.0,58.0,2019-08-08 12:53:00,12:53:00,2019-08-08,31,220,4,12,...,2019-08-08 16:10:06,LH,E,217.943,2019-08-08 15:10:06,15:10:06,31,220,4,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
396,2021-12-27T23:53:00,31.0,92.0,2021-12-27 23:53:00,23:53:00,2021-12-27,52,361,1,23,...,2021-12-28 00:10:09,LH,E,58.443,2021-12-27 23:10:09,23:10:09,52,361,1,23
397,2021-12-28T11:53:00,38.0,70.0,2021-12-28 11:53:00,11:53:00,2021-12-28,52,362,2,11,...,2021-12-28 01:10:03,LH,E,58.055,2021-12-28 00:10:03,00:10:03,52,362,2,0
398,2021-12-29T20:53:00,38.0,93.0,2021-12-29 20:53:00,20:53:00,2021-12-29,52,363,3,20,...,2021-12-29 14:10:01,LH,E,59.199,2021-12-29 13:10:01,13:10:01,52,363,3,13
399,2021-12-30T20:53:00,41.0,100.0,2021-12-30 20:53:00,20:53:00,2021-12-30,52,364,4,20,...,2021-12-30 12:10:05,LH,E,56.524,2021-12-30 11:10:05,11:10:05,52,364,4,11


In [56]:
#US BANK HOLIDAYS
holidays = pd.read_csv('US BANK holidays.csv')
holidays = holidays.drop(columns=['Unnamed: 0']);

mergedData['Date'] = pd.to_datetime(mergedData.Date, format='%Y-%m-%d')
mergedData['Date'] = mergedData['Date'].dt.strftime('%Y-%m-%d')


def holiday(data):

    global add

    add=0
    for x in holidays['date']:

      if (data == holidays.at[add,'date']):
        weekendIndex = 0
        add=0
     #   print("STOP")
        return weekendIndex
      else:
        weekendIndex = 1
        add = add+1
       # print("DROP")

   # print("ROLL")
    add=0
    return weekendIndex

#mergedData['Weekday Index'] = mergedData.apply(lambda x: holiday(x['Date']),axis=1)
mergedData

,DateTime,Temp,Humidity,time_x,Time_x,Date,Week_x,Day_x,Weekday_x,Hour_x,...,END_TIME,BUILDING_ID,TYPE,VALUE,time_y,Time_y,Week_y,Day_y,Weekday_y,Hour_y
0,2019-08-04T12:53:00,77.0,58.0,2019-08-04 12:53:00,12:53:00,2019-08-04,31,216,0,12,...,2019-08-04 18:10:01,LH,E,179.711,2019-08-04 17:10:01,17:10:01,31,216,0,17
1,2019-08-05T14:53:00,79.0,31.0,2019-08-05 14:53:00,14:53:00,2019-08-05,31,217,1,14,...,2019-08-05 18:10:03,LH,E,171.752,2019-08-05 17:10:03,17:10:03,31,217,1,17
2,2019-08-06T14:53:00,82.0,53.0,2019-08-06 14:53:00,14:53:00,2019-08-06,31,218,2,14,...,2019-08-06 15:10:01,LH,E,216.926,2019-08-06 14:10:01,14:10:01,31,218,2,14
3,2019-08-07T12:29:00,77.0,66.0,2019-08-07 12:29:00,12:29:00,2019-08-07,31,219,3,12,...,2019-08-07 13:10:08,LH,E,220.631,2019-08-07 12:10:08,12:10:08,31,219,3,12
4,2019-08-08T12:53:00,78.0,58.0,2019-08-08 12:53:00,12:53:00,2019-08-08,31,220,4,12,...,2019-08-08 16:10:06,LH,E,217.943,2019-08-08 15:10:06,15:10:06,31,220,4,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
396,2021-12-27T23:53:00,31.0,92.0,2021-12-27 23:53:00,23:53:00,2021-12-27,52,361,1,23,...,2021-12-28 00:10:09,LH,E,58.443,2021-12-27 23:10:09,23:10:09,52,361,1,23
397,2021-12-28T11:53:00,38.0,70.0,2021-12-28 11:53:00,11:53:00,2021-12-28,52,362,2,11,...,2021-12-28 01:10:03,LH,E,58.055,2021-12-28 00:10:03,00:10:03,52,362,2,0
398,2021-12-29T20:53:00,38.0,93.0,2021-12-29 20:53:00,20:53:00,2021-12-29,52,363,3,20,...,2021-12-29 14:10:01,LH,E,59.199,2021-12-29 13:10:01,13:10:01,52,363,3,13
399,2021-12-30T20:53:00,41.0,100.0,2021-12-30 20:53:00,20:53:00,2021-12-30,52,364,4,20,...,2021-12-30 12:10:05,LH,E,56.524,2021-12-30 11:10:05,11:10:05,52,364,4,11


In [57]:
#Create weekday index
#0 for Weekdays being 5 or 6 (weekends) OR if it's a holiday
#1 for anything else
def createWeekdayIndex(weekday, date):
    global add
    add = 0


    for x in holidays['date']:
      if (date == holidays.at[add,'date']):
        weekdayIndex = 0
        add=0
        return weekdayIndex
      else:
        weekdayIndex = 1
        add = add+1
    add=0

    if(weekday == 5) or (weekday == 6):
        weekdayIndex = 0
    return weekdayIndex

mergedData['Weekday Index'] = mergedData.apply(lambda x: createWeekdayIndex(x['Weekday_y'],x['Date']),axis=1)
mergedData


,DateTime,Temp,Humidity,time_x,Time_x,Date,Week_x,Day_x,Weekday_x,Hour_x,...,BUILDING_ID,TYPE,VALUE,time_y,Time_y,Week_y,Day_y,Weekday_y,Hour_y,Weekday Index
0,2019-08-04T12:53:00,77.0,58.0,2019-08-04 12:53:00,12:53:00,2019-08-04,31,216,0,12,...,LH,E,179.711,2019-08-04 17:10:01,17:10:01,31,216,0,17,1
1,2019-08-05T14:53:00,79.0,31.0,2019-08-05 14:53:00,14:53:00,2019-08-05,31,217,1,14,...,LH,E,171.752,2019-08-05 17:10:03,17:10:03,31,217,1,17,1
2,2019-08-06T14:53:00,82.0,53.0,2019-08-06 14:53:00,14:53:00,2019-08-06,31,218,2,14,...,LH,E,216.926,2019-08-06 14:10:01,14:10:01,31,218,2,14,1
3,2019-08-07T12:29:00,77.0,66.0,2019-08-07 12:29:00,12:29:00,2019-08-07,31,219,3,12,...,LH,E,220.631,2019-08-07 12:10:08,12:10:08,31,219,3,12,1
4,2019-08-08T12:53:00,78.0,58.0,2019-08-08 12:53:00,12:53:00,2019-08-08,31,220,4,12,...,LH,E,217.943,2019-08-08 15:10:06,15:10:06,31,220,4,15,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
396,2021-12-27T23:53:00,31.0,92.0,2021-12-27 23:53:00,23:53:00,2021-12-27,52,361,1,23,...,LH,E,58.443,2021-12-27 23:10:09,23:10:09,52,361,1,23,1
397,2021-12-28T11:53:00,38.0,70.0,2021-12-28 11:53:00,11:53:00,2021-12-28,52,362,2,11,...,LH,E,58.055,2021-12-28 00:10:03,00:10:03,52,362,2,0,1
398,2021-12-29T20:53:00,38.0,93.0,2021-12-29 20:53:00,20:53:00,2021-12-29,52,363,3,20,...,LH,E,59.199,2021-12-29 13:10:01,13:10:01,52,363,3,13,1
399,2021-12-30T20:53:00,41.0,100.0,2021-12-30 20:53:00,20:53:00,2021-12-30,52,364,4,20,...,LH,E,56.524,2021-12-30 11:10:05,11:10:05,52,364,4,11,1


In [58]:
import sklearn
from sklearn.preprocessing import StandardScaler

features = pd.DataFrame({'Weekday Index':mergedData['Weekday Index'], 'Temp':mergedData['Temp'],'Humidity':mergedData['Humidity']}).dropna()
std_scaler = StandardScaler()
std_scaler

features = pd.DataFrame(std_scaler.fit_transform(features), columns=features.columns)

x_train = np.asarray([features['Weekday Index'],features['Temp'], features['Humidity']]).T
y_train = np.asarray(mergedData['VALUE'])


model = LinearRegression()
#calculate beta using train
model.fit(x_train,y_train)
y_pred = model.predict(x_train)


In [59]:
model.coef_
model.score(x_train,y_train) #This is the R^2 value


0.3216267306070003

In [47]:
import requests
from bs4 import BeautifulSoup

ModuleNotFoundError: No module named 'bs4'

In [ ]:
URL = "https://www.binghamton.edu/academics/academic-calendar.html"
#I can use Wayback machine to get earlier snapshots of the data
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")


In [ ]:
results = soup.find(id='academic-calendar')
#Go to Spring -> Go to Month -> Get Date

#Spring 2022
#Summer 2022
#Fall 2022